### European Central Bank API

This notebook retrieves monthly data on the yield of comparable 10-year government bonds of Eurozone countries from the [ECB API](https://sdw-wsrest.ecb.europa.eu/web/generator/index.html), along with the one year high and low values.

In [1]:
import requests
import pandas as pd

# API in JSON data format returns series index as strings
# tiny function below used to convert to ints during sort
def asint(s):
    return int(s)

In [2]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
clist = ['DE', 'FR', 'IT', 'ES', 'PT', 'GR', 'NL', 'BE', 'AT', 'IE']

d = {}    # Empty dictionary used to store important values
for country in clist:
    key = 'IRS/M.{}.L.L40.CI.0000.EUR.N.Z?startPeriod=2016-01-01'.format(country)
    response = requests.get('{}{}'.format(url,key), headers=headers).json()
    # API returns data series in one dictionary and dates in another
    series = response['dataSets'][0]['series'][u'0:0:0:0:0:0:0:0:0']['observations']
    slist = [data[k][0] for k in sorted(data, key=asint)]
    dates = response['structure']['dimensions']['observation'][0]['values']
    dlist = [i['start'][:10] for i in dates]
    area = response['structure']['dimensions']['series'][1]['values'][0]['name']
    # Convert two lists into pandas dataframe
    df = pd.DataFrame({'date': dlist, area: slist}).set_index('date')
    d[country] = {}     # Create key in dictionary for each country
    d[country]['latest'] = df[area][-1]
    d[country]['label'] = area
    d[country]['1min'] = df[area][-12:].min()  #Most recent year only
    d[country]['1max'] = df[area][-12:].max()
recent_date = pd.to_datetime(df.index[-1]).strftime('%B %Y')  # Latest date as text

In [6]:
pd.DataFrame(d).transpose()

,1max,1min,label,latest
AT,0.5924,0.1068,Austria,0.5924
BE,0.87,0.15,Belgium,0.87
DE,0.35,-0.15,Germany,0.35
ES,1.717,1.008,Spain,1.717
FR,1.03,0.15,France,1.02
GR,9.03,6.94,Greece,7.17
IE,1.06,0.4,Ireland,1.05
IT,2.401,1.176,Italy,2.401
NL,0.486,0.027,Netherlands,0.485
PT,4.043,2.912,Portugal,3.992
